In [1]:
import json
import requests
import networkx as nx
from networkx.readwrite import json_graph
import matplotlib.pyplot as plt
import os

In [2]:
import plotly.plotly as py
import plotly.tools as tls
from plotly.graph_objs import *

mpl_fig = plt.figure()

In [3]:
data_dir = '/Users/christopherallison/.virtualenvs/graph_tools_tests/'

with open(os.path.join(data_dir, 'balanced_tree.json'), "r") as f:
    data = json.load(f)


In [4]:
G = nx.DiGraph()
G = json_graph.node_link_graph(data)

In [5]:
pos=nx.spring_layout(G)

nx.draw_networkx_nodes(G, pos)
nx.draw_networkx_edges(G, pos)

In [20]:
plt.show()

In [21]:
# Write to Graphml format
#nx.write_graphml(G,"story_2015-06_29.graphml")

In [24]:
# add the edges in as disconnected lines in a single trace
edge_trace = Scatter(
    x=[],
    y=[],
    text=[],
    textposition='bottom',
    textfont=Font(
        family='Arial',
    ),
    mode='lines')

for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_trace['x'] += [x0, x1, None]
    edge_trace['y'] += [y0, y1, None]
    
# add the nodes in as a scatter
node_trace = Scatter(
    x=[], 
    y=[], 
    mode='markers+text',
    name=[],
    text=[],
    textposition='top center',
    textfont=Font(
        family='Arial',
    ),
    marker=Marker(size=[]))

for node in G.nodes():
    if node:
        x, y = pos[node]
        node_trace['x'].append(x)
        node_trace['y'].append(y)


In [25]:
# size the node points by the number of connections
for node in G.nodes():
    node_trace['marker']['size'] = 10

In [26]:
fig = Figure(data=Data([edge_trace, node_trace]),
             layout=Layout(title='Balanced Tree Graph from Networkx', plot_bgcolor="rgb(217, 217, 217)",
                           showlegend=True, xaxis=XAxis(showgrid=False, zeroline=False, showticklabels=False),
                           yaxis=YAxis(showgrid=False, zeroline=False, showticklabels=False)))

# send the figure to Plotly and embed an iframe in this notebook
py.iplot(fig, filename='balanced_tree_graph')

In [7]:

import seaborn as sns
import numpy as np

# Add a dark background
sns.set_style('dark')

# Initialize figure
fig = plt.figure(figsize = (8, 8))
ax  = fig.add_subplot(1, 1, 1)

# Create force-directed layout for the graph
pos = nx.spring_layout(G)
pos_array = np.array(pos.values()).T # Format into a NumPy array

# Define node colors based on in-degree
node_color = np.array([len(G.predecessors(v)) for v in G], dtype=float)

# Define node sizes to be proportional to PageRank values
sizes = 1E4*nx.pagerank(G).values()

# Plot each edge
for i in G:
    for j in G[i]:
        # Compute in-marker size
        # The next few lines just resize the
        # incoming markers to look decent
        # with the varying node sizes
        p = pos_array[:, j] - pos_array[:, i]
        s = -.4*(1 + .4*np.log(sizes[j]/sizes.min()))
        p = s*p/np.linalg.norm(p) + pos_array[:, j]

        # Plot markers
        ax.plot([pos_array[0, j], p[0]], [pos_array[1, j], p[1]], '-k', alpha = .5, linewidth = 2, zorder = 9)

        #Plot edges
        ax.plot(pos_array[0, [i, j]], pos_array[1, [i, j]], '-w')

# Plot nodes
ax.scatter(pos_array[0], pos_array[1], sizes, color = cm.coolwarm(node_color/node_color.max()), zorder = 10)

# Remove axis ticks
pp.xticks([])
pp.yticks([])

AttributeError: 'Graph' object has no attribute 'predecessors'

In [9]:
nx.write_gexf(G, 'gephi_balanced_tree.gexf')